In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def base_lagrangian_basis(q, e):
    nqpts = q.shape[0]
    nepts = e.shape[0]
    
    basis = np.ones((nepts, nqpts), dtype=np.float64)
    for i in range(nepts):
        for j in range(nqpts):
            for k in range(nqpts):
                if k == j:
                    continue
                basis[i, j] *= (e[i] - q[k])/(q[j] - q[k])
    
    return basis

def base_lagrangian_basis2d(q_x, q_y, e_x, e_y):
    nqxpts = q_x.shape[0]
    nqypts = q_y.shape[0]
    nexpts = e_x.shape[0]
    neypts = e_y.shape[0]
    
    #basis_shape = (nexpts, neypts, nqxpts, nqypts)
    #basis = np.empty(basis_shape)
    b1 = base_lagrangian_basis(q_x, e_x)
    b2 = base_lagrangian_basis(q_y, e_y)
    
    """
    for i in range(nexpts):
        for j in range(neypts):
            for k in range(nqxpts):
                for ll in range(nqypts):
                    basis[i, j, k, ll] = b1[i, k] * b2[j, ll]
    """
    
    b1 = b1.reshape((nexpts, 1, nqxpts, 1))
    b2 = b2.reshape((1, neypts, 1, nqypts))
    basis = b1 * b2
    
    return basis

def base_interpolate1d(f, q, e):
    nqpts = q.shape[0]
    nepts = e.shape[0]
    
    basis = base_lagrangian_basis(q, e)
    
    f_e = np.zeros((nepts), dtype=np.float64)
    for i in range(nepts):
        for j in range(nqpts):
            f_e[i] += basis[i, j] * f(q[j])
            
    return f_e

def base_interpolate2d(f, q_x, q_y, e_x, e_y):
    nqxpts = q_x.shape[0]
    nqypts = q_y.shape[0]
    nexpts = e_x.shape[0]
    neypts = e_y.shape[0]
    
    basis = base_lagrangian_basis2d(q_x, q_y, e_x, e_y)
    f_e = np.zeros((nexpts, neypts), dtype=np.float64)
    
    f_q = f_eval2d(f, q_x, q_y)
    """
    for i in range(nexpts):
        for j in range(neypts):
            for k in range(nqxpts):
                for ll in range(nqypts):
                    f_e[i, j] += basis[i, j, k, ll] *f_q[k, ll]
    """
    """
    #get rid of implicit matrix mult
    for k in range(nqxpts):
        for ll in range(nqypts):
            f_e[:, :] += basis[:, :, k, ll] * f_q[k, ll]
    """
    
    basis_reshaped = basis.reshape((nexpts*neypts, nqxpts*nqypts))
    f_q_reshaped = f_q.reshape((nqxpts*nqypts))
    
    f_e = basis_reshaped @ f_q_reshaped
    f_e = f_e.reshape((nexpts, neypts))
    
    return f_e


In [3]:
def f1d(x):
    return np.sin(2 * np.pi * x)

def f2d(x, y):
    return np.sin(2 * np.pi * x) + y * y

def f_eval1d(f, e):
    ret = np.empty((e.shape[0]), dtype=np.float64)
    for (i, x) in enumerate(e):
        ret[i] = f(x)
    
    return ret

def f_eval2d(f, ex, ey):
    ret = np.empty((ex.shape[0], ey.shape[0]), dtype=np.float64)
    
    for (i, x) in enumerate(ex):
        for (j, y) in enumerate(ey):
            ret[i, j] = f(x, y)
    
    return ret
    #return np.array([[f(x, y) for x in] for x in ex], dtype=np.float64)

#|e| * |q|: |e| evaluations of the |q| polynomials
def lagrangian_basis(q, e):
    #|q| * (|q|-1)
    reduced = np.array([[qj for (j, qj) in enumerate(q) if i != j] for (i, qi) in enumerate(q)], dtype=np.float64)

    #qr = q.reshape((q.shape[0], 1))
    #denoms = np.prod(qr - reduced, axis=1)
    #[|q| * 1->(|q|-1)] - [|q| * (|q|-1|)] -> [|q| * (|q|-1)]
    #sum along axis 1 [|q| * (|q|-1)] -> |q|
    denoms = np.prod(q[:, None] - reduced, axis=1)
    
    #er = e.reshape((e.shape[0], 1, 1))
    #basis = np.prod(er - reduced, axis=2) / denoms
    #[|e| * 1->|q| * 1->(|q|-1)] - [1->|e| * |q| * (|q|-1)] -> [|e| * |q| * (|q|-1)]
    #product along axis 2 [|e| * |q| * (|q|-1)] -> [|e| * |q|]
    #division [|e| * |q|] / [1->|e| * |q|] -> [|e| * |q|]
    basis = np.prod(e[:, None, None] - reduced, axis=2) / denoms
    
    return basis

def lagrangian_basisV2(q, e):
    reduced = np.array([[qj for (j, qj) in enumerate(q) if i != j] for (i, qi) in enumerate(q)], dtype=np.float64)
    
    denoms = np.prod(q[:, None] - reduced, axis=1)
    
    basis = np.ones((e.shape[0], q.shape[0]), dtype=np.float64) / denoms
    for i in range(basis.shape[0]):
        basis[i, :] *= np.prod(e[i, None, None] - reduced, axis=1)
    
    return basis

#|ex| * |ey| * |qx| * |qY|: |ex| * |ey| evaluations of the |qx| * |qY| polynomials
def lagrangian_basis2d(qx, qy, ex, ey):
    """
    qxlen = qx.shape[0]
    qylen = qy.shape[0]
    exlen = ex.shape[0]
    eylen = ey.shape[0]
    
    #|ex| * |qx| -> |ex| * 1 * |qx| * 1 
    bx = lagrangian_basis(qx, ex).reshape((exlen, 1, qxlen, 1))
    #|ey| * |qy| -> 1 * |ey| * 1 * |qy|
    by = lagrangian_basis(qy, ey).reshape((1, eylen, 1, qylen))
    
    #|ex| * |ey| * |qx| * |qY| because of broadcasting
    return bx * by
    """
    return lagrangian_basis(qx, ex)[:, None, :, None] * lagrangian_basis(qy, ey)[None, :, None, :]

def lagrangian_basis2dV2(qx, qy, ex, ey):
    return lagrangian_basisV2(qx, ex)[:, None, :, None] * lagrangian_basisV2(qy, ey)[None, :, None, :]

def interpolate1d(f, q, e):
    return lagrangian_basis(q, e) @ f_eval1d(f, q)

def interpolate2d(f, qx, qy, ex, ey):
    qxlen = qx.shape[0]
    qylen = qy.shape[0]
    exlen = ex.shape[0]
    eylen = ey.shape[0]
    
    #|qx| * |qY| * |ex| * |ey|
    basis = lagrangian_basis2d(qx, qy, ex, ey)
    #|qx X qY| * |ex X ey|
    basis = basis.reshape((exlen * eylen, qxlen * qylen)) 
    
    #|qx| * |qY|
    fq = f_eval2d(f2d, qx, qy)
    #|qx X qY|
    fq = fq.reshape((qxlen * qylen))
    
    #|ex| * |ey|
    #fe = np.zeros((exlen, eylen), dtype=np.float64)
    
    #|ex X ey|
    fe = basis @ fq
    #|ex| * |ey|
    fe = fe.reshape((exlen, eylen))
    
    return fe

def interpolate2dV2(f, qx, qy, ex, ey):
    qxlen = qx.shape[0]
    qylen = qy.shape[0]
    exlen = ex.shape[0]
    eylen = ey.shape[0]
    
    basis = lagrangian_basis2dV2(qx, qy, ex, ey)
    basis = basis.reshape((exlen * eylen, qxlen * qylen))
    
    fq = f_eval2d(f2d, qx, qy)
    fq = fq.reshape((qxlen * qylen))
    
    fe = basis @ fq
    fe = fe.reshape((exlen, eylen))
    
    return fe

In [10]:
#evaluation coords
exlen = eylen = 1025
ex = np.linspace(0, 1, exlen)
ey = np.linspace(0, 1, eylen)

#points for interpolation
qxlen = qylen = 18
qx = np.linspace(0, 1, qxlen)
qy = np.linspace(0, 1, qylen)

fe = f_eval2d(f2d, ex, ey)

#1d
"""
fex = f_eval1d(f1d, ex)

base_xinterp = base_interpolate1d(f1d, qx, ex)
xinterp = interpolate1d(f1d, qx, ex)

print(f"xinterp delta max: {np.max(np.abs(xinterp-base_xinterp))}")
"""
#2d
#base_basis = base_lagrangian_basis2d( qx, qy, ex, ey)
#basis = lagrangian_basis2d( qx, qy, ex, ey)
#print(f"basis delta max: {np.max(np.abs(basis-base_basis))}")

#base_interp = base_interpolate2d(f2d, qx, qy, ex, ey)

interp = interpolate2d(f2d, qx, qy, ex, ey)
#print(f"interp delta max: {np.max(np.abs(interp - base_interp))}")

err = np.max(np.abs(fe - interp))
mse = np.sum(np.power(fe - interp, 2)) / np.prod(fe.shape)
print(f"with {qxlen}*{qylen} sample points: err: {err} mse: {mse}")

c = """
interpV2 = interpolate2dV2(f2d, qx, qy, ex, ey)
print(f"interp delta max: {np.max(np.abs(interpV2 - base_interp))}")

err = np.max(np.abs(fe - interpV2))
mse = np.sum(np.power(fe - interpV2, 2)) / np.prod(fe.shape)
print(f"V2: with {qxlen}*{qylen} sample points: err: {err} mse: {mse}")
"""


with 18*18 sample points: err: 7.592550199664316e-11 mse: 9.233615682098931e-24


In [5]:
%timeit base_lagrangian_basis(qx, ex)

338 ms ± 9.46 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%timeit lagrangian_basis(qx, ex)

1.09 ms ± 24.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [7]:
%timeit base_interp = base_interpolate2d(f2d, qx, qy, ex, ey)

1.61 s ± 20.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
%timeit interp = interpolate2d(f2d, qx, qy, ex, ey)
%timeit interp = interpolate2dV2(f2d, qx, qy, ex, ey)

973 ms ± 52 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.08 s ± 124 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [32]:
%prun interp = interpolate2dV2(f2d, qx, qy, ex, ey)

In [68]:
fe = f_eval2d(f2d, ex, ey)
diff = np.max(np.abs(interp - base_interp))
base_err = np.max(np.abs(fe - base_interp))
err = np.max(np.abs(fe - interp))

#print(f"with {qxlen}*{qylen} sample points: diff: {diff} base_err: {base_err} err: {err}")
print(f"with {qxlen}*{qylen} sample points: err: {err}")

with 19*19 sample points: err: 2.7009638969843763e-09


In [ ]:
_ = plot()